# *Ajustando Hiperparamentos

## Escolhemos os melhores hiperparâmetros - Grid Search. 


Vamos aprender algo que vai economizar o tempo enquant estivermos treinando modelos, nós vamos usar a técnica chamada Grid Search. 

que é uma técnia que nos permite teestar exaustivamente vários parâmetros possíveis para o modelo prophet de forma automatizada, ganhando aqui muito tempo com a experimentação de parâmetros. 

O prophet não possui um método próprio para fazer o grid search, igual ao escalar, mas é muito simples para criarmos um. 

Então, como primeiro passo, será necessário criar um dicionário onde cada chave será o nome de um parâmetro do prophet e como o valor nós passamos uma lista com todas as opções que a gente deseja testar. 

In [1]:
param_grid = {'changepoint_prior_scale': [0.5, 0.1, 0.01,0.001],
            'seasonality_prior_scale': [10.0, 1.0, 0.1,0.01],
            'seasonality_mode': ['additive','multiplicative']}


No segundo bloco nós vamos utilizar o pacote itertools do python, que vai iterar nesse dicionário, e criar uma lista com cada uma das combinações de valores possíveis. Além de importar o itertools, importamos o numpy.

In [2]:
import numpy as np
import itertools
all_params = [dict(zip(param_grid.keys(), value))
            for value in itertools.product(
                *param_grid.values())]

In [5]:
all_params

[{'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 10.0,
  'seasonality_mode': 'additive'},
 {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 10.0,
  'seasonality_mode': 'multiplicative'},
 {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 1.0,
  'seasonality_mode': 'additive'},
 {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 1.0,
  'seasonality_mode': 'multiplicative'},
 {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 0.1,
  'seasonality_mode': 'additive'},
 {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 0.1,
  'seasonality_mode': 'multiplicative'},
 {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 0.01,
  'seasonality_mode': 'additive'},
 {'changepoint_prior_scale': 0.5,
  'seasonality_prior_scale': 0.01,
  'seasonality_mode': 'multiplicative'},
 {'changepoint_prior_scale': 0.1,
  'seasonality_prior_scale': 10.0,
  'seasonality_mode': 'additive'},
 {'changepoint_prior_scale': 0.1,
  'seasona


Esse comando que faz nós criarmos uma lista de parâmetros exclusivos com base em todas as combinações possíveis desse dicionário aqui acima, 











In [4]:
rmse_values = []


Nós também estamos criando uma lista vazia que será responsável por armazenar os valores das métricas de RMSE para cada execução. 

Considerando que essa foia a métrica da nossa escolha para avaliar o modelo.

Assim, ficou esse dicionário ao params, nós vemos que ele é uma coleção de todas as combinações que temos. 

### Vamos iterar sobre cada uma das 32 combinações que foram geradas. 

In [6]:
len(all_params)

32

In [12]:
import pandas as pd
from prophet import Prophet
df = pd.read_csv('../Modulo11/online_retail.csv')
display(df)
df.columns = ['ds', 'y']
display(df)

/home/fabiene/anaconda3/envs/Udemy/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


,date,total sales
0,2009-12-01,3106.000000
1,2009-12-02,3254.000000
2,2009-12-03,2951.000000
3,2009-12-04,2529.000000
4,2009-12-05,2644.541758
...,...,...
1099,2012-12-04,2726.403434
1100,2012-12-05,4528.532962
1101,2012-12-06,6125.495419
1102,2012-12-07,3476.642731


,ds,y
0,2009-12-01,3106.000000
1,2009-12-02,3254.000000
2,2009-12-03,2951.000000
3,2009-12-04,2529.000000
4,2009-12-05,2644.541758
...,...,...
1099,2012-12-04,2726.403434
1100,2012-12-05,4528.532962
1101,2012-12-06,6125.495419
1102,2012-12-07,3476.642731


In [15]:
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

for params in all_params:
    model = Prophet(yearly_seasonality=4, **params).fit(df)
    df_cv = cross_validation(model,
                             horizon='30 days',
                             parallel='processes')
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmse_values.append(df_p['rmse'].values[0])


21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] done processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] start processing
21:39:50 - cmdstanpy - INFO - Chain [1] done processing
21:39:50 - cmdstanpy - INFO - Chain [1] done processing
21:39:50 - cmdstanpy - INFO - Chain [1] done processing
21:39:50 - cmdstanpy - INFO - Chain

Ou seja, teremos 32 modelos diferentes testados com configurações diferentes.  Repare que nós estamos instanciando e passando aqui dinamicamente as nossas configurações. Em seguida, nós estamos fazendo a avalição via cross_validation desse modelo em seguida, estamos gerando as métricas de performance. E com isso coletamos a nossa métrica escolhida, que é o rmse armazenamos na nossa lista de valores. 

Como nós vamos testar 32 modelos de séries temporais, esse comando pode demorar um pouco.

Terminando as 32 iterações. 

Agora vamos ver como identificar qual que foi o melhor conjunto de parâmetros. Aqui nós temos um dataframe que foi criado atribuindo a cada combinação qual que foi o seu valor de métrica correspondente, para pegar de forma simplificada qual foi o melhor conjunto de parâmetros

In [16]:
results = pd.DataFrame(all_params)
results['rmse'] = rmse_values
results.head()

,changepoint_prior_scale,seasonality_prior_scale,seasonality_mode,rmse
0,0.5,10.0,additive,783.702544
1,0.5,10.0,multiplicative,924.563298
2,0.5,1.0,additive,794.519150
3,0.5,1.0,multiplicative,897.914473
4,0.5,0.1,additive,764.905889


In [17]:
best_params = all_params[np.argmin(rmse_values)]
print(best_params)

{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 10.0, 'seasonality_mode': 'multiplicative'}


usando a função argmin do numpy que vai retornar o índice do dicionário para qual os parâmetros possuem o menor RMSE, muito bem, estão esse foi o melhor conjunto de parâmetros dados todas aquelas combinações que nós testamos. E essa é a forma mais produtiva que você vai ver de testar os parâmetros no Prophet. 